Используемые библиотеки

# Дифференциальные уравнения первого порядка

Задача 1.5

Одним из основных законов теории скоростей химических реакций является закон действующих масс, согласно которому скорость химической реакции при постоянной температуре пропорциональна произведению концентраций веществ, участвующих в данный момент в реакции.

Решим следующую задачу. Два жидких химических вещества $A$ и $B$ объемом $V_{a}$ и $V_{b}$ соответственно в процессе химической реакции образуют новое жидкое химическое вещество $C$. Полагая, что температура в процессе реакции не меняется, а также что из каждых двух объемов вещества $A$ и одного объема вещества $B$ образуется три объема вещества $C$, получаем зависимость количества вещества $C$ от времени в виде дифференциального уравнения $\frac{dx}{dt} = k (V_{a} - \frac{2}{3} * x)(V_{b} - \frac{1}{3} * x)$, где $x$ - объем (в литрах) вещества $C$, $k$ - положительный коэффициент пропорционально равный $0.02$. Определить время для полного прекращения реакции, а также объем вещества $A$ и $B$ в момент прекращения реакции.

# Системы дифференциальных уравнений

Задача 2.5

Построить фазовый портрет системы дифференциальных уравнений 

$$
\{\begin{matrix}
 \frac{dx}{dt} = -y - x * (x^{2} + y^{2} - 1) \\
 \frac{dy}{dt} = x - y * (x^{2} + y^{2} - 1)
\end{matrix}
$$

# Равномерно распределенная дискретная случайная величина

Задача 3.5

Рассмотрим задачу о пьяном прохожем, которую называют еще задачей о случайном блуждании. Предположим, что пьяный, стоя на углу улицы, решает прогуляться. Пусть вероятности того, что, достигнув очередного перекрестка, он пойдет на север, юг, восток или запад, одинаковы. Оценить вероятность того, что, пройдя 10 кварталов, пьяный окажется не далее двух кварталов от места, где он начал прогулку. Оценку вероятности провести на основании 1000 испытаний.


# Равномерно распределенная непрерывная случайная величина

Задача 5.5

Оценить интеграл $I = \int^{\pi}_{0}\sin{x*dx}$ (как показывают элементарные вычисления ), используя метод статистических испытаний, при объемах выборки $n = 10, 20, 50, 100, 1000, 10000$.


# Нормально распределенная случайная величина
